In [1]:
import numpy as np
from torchvision.datasets import MNIST 

def download_mnist(is_train: bool):
    dataset = MNIST(
        root='./data',
        train=is_train,
        transform=lambda x: np.array(x).flatten(),
        download=False
    )

    mnist_data = []
    mnist_labels = []
    for image,label in dataset:
        mnist_data.append(image)
        mnist_labels.append(label)

    return np.array(mnist_data), np.array(mnist_labels)

**Download data sets:**
 
*train_X* = imagini de antrenament

*train_Y* = labels pentru imagini de antrenament

*test_X* = imagini pentru testare

*test_Y* = labels pentru imagini de test

In [2]:
train_X, train_Y = download_mnist(True)

/tmp/ipykernel_79674/3851143839.py:8: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  transform=lambda x: np.array(x).flatten(),


In [3]:
test_X, test_Y = download_mnist(False)

/tmp/ipykernel_79674/3851143839.py:8: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  transform=lambda x: np.array(x).flatten(),


**2.** Normalize the data and convert the labels to *one-hot-encoding*

In [4]:
train_X_norm = train_X / 255
test_X_norm = test_X / 255

In [5]:
from sklearn.preprocessing import OneHotEncoder 

enc = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
train_Y_enc = enc.fit_transform(train_Y.reshape(-1,1))
test_Y_enc = enc.fit_transform(test_Y.reshape(-1,1))

In [6]:
def split_into_batches(trainset,labels,batches_dimension):
    num_batches = int(np.ceil(len(trainset) / batches_dimension))
    
    batches = []
    for i in range(num_batches):
        start_index = i * batches_dimension
        end_index = min((i + 1) * batches_dimension, len(trainset))
        batch_data = trainset[start_index:end_index]
        batch_labels = labels[start_index:end_index]
        batches.append((batch_data, batch_labels))
    
    return batches

In [7]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)

def forward_propagation(X,W,b):
    Z = np.dot(X,W) + b
    A = softmax(Z)
    return A

def compute_loss(A,Y):
    m = Y.shape[0]
    log_likelihood = -np.log(A[range(m), Y.argmax(axis=1)])
    return np.sum(log_likelihood) / m

def backward_propagation(X,Y,A):
    m = X.shape[0]
    dz = A - Y
    dW = np.dot(X.T, dz) / m
    db = np.sum(dz, axis=0, keepdims=True) / m
    return dW, db

In [8]:
def train(epoch,train_data, w, b):
    X_batch, Y_batch = train_data
    A = forward_propagation(X_batch, w, b)
    new_w, new_b = backward_propagation(X_batch,Y_batch,A)

    return new_w, new_b

In [9]:
def update_params(W,b,dW,db,learning_rate):
    W -= learning_rate * dW
    b -= learning_rate * db
    return W, b

In [10]:
Weight = np.random.randn(784,10) * 0.01
bias = np.zeros((1,10))

In [15]:
import concurrent.futures
from concurrent.futures import as_completed

for epoch in range(150):
    schuffle = np.random.permutation(train_X_norm.shape[0])
    train_X_norm = train_X_norm[schuffle]
    train_Y_enc = train_Y_enc[schuffle]

    batches = split_into_batches(train_X_norm,train_Y_enc,100)

    results = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = {executor.submit(train, epoch, batch, Weight, bias): batch for batch in batches}

        for future in as_completed(futures):
            result = future.result()
            Weight, bias = update_params(Weight,bias,result[0],result[1],0.01)
            
    print(f'Epoch {epoch} done.')


Epoch 0 done.
Epoch 1 done.
Epoch 2 done.
Epoch 3 done.
Epoch 4 done.
Epoch 5 done.
Epoch 6 done.
Epoch 7 done.
Epoch 8 done.
Epoch 9 done.
Epoch 10 done.
Epoch 11 done.
Epoch 12 done.
Epoch 13 done.
Epoch 14 done.
Epoch 15 done.
Epoch 16 done.
Epoch 17 done.
Epoch 18 done.
Epoch 19 done.
Epoch 20 done.
Epoch 21 done.
Epoch 22 done.
Epoch 23 done.
Epoch 24 done.
Epoch 25 done.
Epoch 26 done.
Epoch 27 done.
Epoch 28 done.
Epoch 29 done.
Epoch 30 done.
Epoch 31 done.
Epoch 32 done.
Epoch 33 done.
Epoch 34 done.
Epoch 35 done.
Epoch 36 done.
Epoch 37 done.
Epoch 38 done.
Epoch 39 done.
Epoch 40 done.
Epoch 41 done.
Epoch 42 done.
Epoch 43 done.
Epoch 44 done.
Epoch 45 done.
Epoch 46 done.
Epoch 47 done.
Epoch 48 done.
Epoch 49 done.
Epoch 50 done.
Epoch 51 done.
Epoch 52 done.
Epoch 53 done.
Epoch 54 done.
Epoch 55 done.
Epoch 56 done.
Epoch 57 done.
Epoch 58 done.
Epoch 59 done.
Epoch 60 done.
Epoch 61 done.
Epoch 62 done.
Epoch 63 done.
Epoch 64 done.
Epoch 65 done.
Epoch 66 done.
Epoch

In [12]:
def predict(X,W,b):
    A = forward_propagation(X,W,b)
    return np.argmax(A, axis=1)

In [17]:
from sklearn.metrics import accuracy_score

train_predictions = predict(train_X_norm,Weight,bias)
test_predictions = predict(test_X_norm,Weight,bias)

print(f'Training Accuracy: {accuracy_score(np.argmax(train_Y_enc,axis=1), train_predictions) * 100:.2f}%')
print(f'Testing Accuracy: {accuracy_score(test_Y, test_predictions) * 100:.2f}%')

Training Accuracy: 92.82%
Testing Accuracy: 92.40%
